In [1]:
from itertools import count

import polars as pl
from great_tables import GT, html, loc, md, style
from great_tables._helpers import px
from great_tables.data import gtcars as gtcars_pd

# pl.Config.set_tbl_rows(100)

In [2]:
gtcars = pl.from_pandas(gtcars_pd)
gtcars.head()

mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,ctry_origin,msrp
str,str,f64,str,str,f64,f64,f64,f64,f64,f64,str,str,str,f64
"""Ford""","""GT""",2017.0,"""Base Coupe""","""coupe""",647.0,6250.0,550.0,5900.0,11.0,18.0,"""rwd""","""7a""","""United States""",447000.0
"""Ferrari""","""458 Speciale""",2015.0,"""Base Coupe""","""coupe""",597.0,9000.0,398.0,6000.0,13.0,17.0,"""rwd""","""7a""","""Italy""",291744.0
"""Ferrari""","""458 Spider""",2015.0,"""Base""","""convertible""",562.0,9000.0,398.0,6000.0,13.0,17.0,"""rwd""","""7a""","""Italy""",263553.0
"""Ferrari""","""458 Italia""",2014.0,"""Base Coupe""","""coupe""",562.0,9000.0,398.0,6000.0,13.0,17.0,"""rwd""","""7a""","""Italy""",233509.0
"""Ferrari""","""488 GTB""",2016.0,"""Base Coupe""","""coupe""",661.0,8000.0,561.0,3000.0,15.0,22.0,"""rwd""","""7a""","""Italy""",245400.0


In [3]:
cast_dict = {
    "year": pl.UInt16,
    "hp": pl.UInt32,
    "hp_rpm": pl.UInt32,
    "trq": pl.UInt32,
    "trq_rpm": pl.UInt32,
    "mpg_c": pl.UInt8,
    "mpg_h": pl.UInt8,
    "msrp": pl.UInt32,
}

gtcars = gtcars.cast(cast_dict)
gtcars.head()

mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,ctry_origin,msrp
str,str,u16,str,str,u32,u32,u32,u32,u8,u8,str,str,str,u32
"""Ford""","""GT""",2017,"""Base Coupe""","""coupe""",647,6250,550,5900,11,18,"""rwd""","""7a""","""United States""",447000
"""Ferrari""","""458 Speciale""",2015,"""Base Coupe""","""coupe""",597,9000,398,6000,13,17,"""rwd""","""7a""","""Italy""",291744
"""Ferrari""","""458 Spider""",2015,"""Base""","""convertible""",562,9000,398,6000,13,17,"""rwd""","""7a""","""Italy""",263553
"""Ferrari""","""458 Italia""",2014,"""Base Coupe""","""coupe""",562,9000,398,6000,13,17,"""rwd""","""7a""","""Italy""",233509
"""Ferrari""","""488 GTB""",2016,"""Base Coupe""","""coupe""",661,8000,561,3000,15,22,"""rwd""","""7a""","""Italy""",245400


## Row groups

### Get 2 rows from each group

In [4]:
gtcars_8 = (
    gtcars.sort("ctry_origin")
    .select(pl.col("ctry_origin"), pl.all().exclude("ctry_origin"))
    .filter(pl.col("ctry_origin").ne("United Kingdom"))
    .filter(pl.int_range(pl.len()).over("ctry_origin").lt(2))
)
gtcars_8

ctry_origin,mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,msrp
str,str,str,u16,str,str,u32,u32,u32,u32,u8,u8,str,str,u32
"""Germany""","""BMW""","""6-Series""",2016,"""640 I Coupe""","""coupe""",315,5800,330,1400,20,30,"""rwd""","""8am""",77300
"""Germany""","""BMW""","""i8""",2016,"""Mega World Coupe""","""coupe""",357,5800,420,3700,28,29,"""awd""","""6am""",140700
"""Italy""","""Ferrari""","""458 Speciale""",2015,"""Base Coupe""","""coupe""",597,9000,398,6000,13,17,"""rwd""","""7a""",291744
"""Italy""","""Ferrari""","""458 Spider""",2015,"""Base""","""convertible""",562,9000,398,6000,13,17,"""rwd""","""7a""",263553
"""Japan""","""Acura""","""NSX""",2017,"""Base Coupe""","""coupe""",573,6500,476,2000,21,22,"""awd""","""9a""",156000
"""Japan""","""Nissan""","""GT-R""",2016,"""Premium Coupe""","""coupe""",545,6400,436,3200,16,22,"""awd""","""6a""",101770
"""United States""","""Ford""","""GT""",2017,"""Base Coupe""","""coupe""",647,6250,550,5900,11,18,"""rwd""","""7a""",447000
"""United States""","""Chevrolet""","""Corvette""",2016,"""Z06 Coupe""","""coupe""",650,6400,650,3600,15,22,"""rwd""","""7m""",88345


### Sort

In [5]:
gtcars_8_sort_by = (
    gtcars.sort("ctry_origin")
    .select(pl.col("ctry_origin"), pl.all().exclude("ctry_origin"))
    .filter(pl.col("ctry_origin").ne("United Kingdom"))
    .filter(pl.int_range(pl.len()).over("ctry_origin").lt(2))
    .select(
        pl.all().sort_by(["mfr", "msrp"], descending=[False, True]).over("ctry_origin")
    )
)
gtcars_8_sort_by

ctry_origin,mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,msrp
str,str,str,u16,str,str,u32,u32,u32,u32,u8,u8,str,str,u32
"""Germany""","""BMW""","""i8""",2016,"""Mega World Coupe""","""coupe""",357,5800,420,3700,28,29,"""awd""","""6am""",140700
"""Germany""","""BMW""","""6-Series""",2016,"""640 I Coupe""","""coupe""",315,5800,330,1400,20,30,"""rwd""","""8am""",77300
"""Italy""","""Ferrari""","""458 Speciale""",2015,"""Base Coupe""","""coupe""",597,9000,398,6000,13,17,"""rwd""","""7a""",291744
"""Italy""","""Ferrari""","""458 Spider""",2015,"""Base""","""convertible""",562,9000,398,6000,13,17,"""rwd""","""7a""",263553
"""Japan""","""Acura""","""NSX""",2017,"""Base Coupe""","""coupe""",573,6500,476,2000,21,22,"""awd""","""9a""",156000
"""Japan""","""Nissan""","""GT-R""",2016,"""Premium Coupe""","""coupe""",545,6400,436,3200,16,22,"""awd""","""6a""",101770
"""United States""","""Chevrolet""","""Corvette""",2016,"""Z06 Coupe""","""coupe""",650,6400,650,3600,15,22,"""rwd""","""7m""",88345
"""United States""","""Ford""","""GT""",2017,"""Base Coupe""","""coupe""",647,6250,550,5900,11,18,"""rwd""","""7a""",447000


### Preferred group order

In [6]:
order_countries = dict(
    zip(("Germany", "Italy", "United States", "Japan", "United Kingdom"), count())
) # {'Germany': 0, 'Italy': 1, 'United States': 2, 'Japan': 3, 'United Kingdom': 4}

gtcars_8_pref_group_order = (
    gtcars.sort(pl.col("ctry_origin").replace(order_countries, return_dtype=pl.UInt8))
    .select(pl.col("ctry_origin"), pl.all().exclude("ctry_origin"))
    .filter(pl.col("ctry_origin").ne("United Kingdom"))
    .filter(pl.int_range(pl.len()).over("ctry_origin").lt(2))
    .select(
        pl.all().sort_by(["mfr", "msrp"], descending=[False, True]).over("ctry_origin")
    )
)
gtcars_8_pref_group_order

ctry_origin,mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,msrp
str,str,str,u16,str,str,u32,u32,u32,u32,u8,u8,str,str,u32
"""Germany""","""BMW""","""i8""",2016,"""Mega World Coupe""","""coupe""",357,5800,420,3700,28,29,"""awd""","""6am""",140700
"""Germany""","""BMW""","""6-Series""",2016,"""640 I Coupe""","""coupe""",315,5800,330,1400,20,30,"""rwd""","""8am""",77300
"""Italy""","""Ferrari""","""458 Speciale""",2015,"""Base Coupe""","""coupe""",597,9000,398,6000,13,17,"""rwd""","""7a""",291744
"""Italy""","""Ferrari""","""458 Spider""",2015,"""Base""","""convertible""",562,9000,398,6000,13,17,"""rwd""","""7a""",263553
"""United States""","""Chevrolet""","""Corvette""",2016,"""Z06 Coupe""","""coupe""",650,6400,650,3600,15,22,"""rwd""","""7m""",88345
"""United States""","""Ford""","""GT""",2017,"""Base Coupe""","""coupe""",647,6250,550,5900,11,18,"""rwd""","""7a""",447000
"""Japan""","""Acura""","""NSX""",2017,"""Base Coupe""","""coupe""",573,6500,476,2000,21,22,"""awd""","""9a""",156000
"""Japan""","""Nissan""","""GT-R""",2016,"""Premium Coupe""","""coupe""",545,6400,436,3200,16,22,"""awd""","""6a""",101770


In [7]:
GT(
    gtcars_8_pref_group_order, groupname_col="ctry_origin", rowname_col="mfr"
).tab_stubhead(label="mfr")

GT(_tbl_data=shape: (8, 15)
┌───────────────┬───────────┬──────────────┬──────┬───┬───────┬────────────┬───────┬────────┐
│ ctry_origin   ┆ mfr       ┆ model        ┆ year ┆ … ┆ mpg_h ┆ drivetrain ┆ trsmn ┆ msrp   │
│ ---           ┆ ---       ┆ ---          ┆ ---  ┆   ┆ ---   ┆ ---        ┆ ---   ┆ ---    │
│ str           ┆ str       ┆ str          ┆ u16  ┆   ┆ u8    ┆ str        ┆ str   ┆ u32    │
╞═══════════════╪═══════════╪══════════════╪══════╪═══╪═══════╪════════════╪═══════╪════════╡
│ Germany       ┆ BMW       ┆ i8           ┆ 2016 ┆ … ┆ 29    ┆ awd        ┆ 6am   ┆ 140700 │
│ Germany       ┆ BMW       ┆ 6-Series     ┆ 2016 ┆ … ┆ 30    ┆ rwd        ┆ 8am   ┆ 77300  │
│ Italy         ┆ Ferrari   ┆ 458 Speciale ┆ 2015 ┆ … ┆ 17    ┆ rwd        ┆ 7a    ┆ 291744 │
│ Italy         ┆ Ferrari   ┆ 458 Spider   ┆ 2015 ┆ … ┆ 17    ┆ rwd        ┆ 7a    ┆ 263553 │
│ United States ┆ Chevrolet ┆ Corvette     ┆ 2016 ┆ … ┆ 22    ┆ rwd        ┆ 7m    ┆ 88345  │
│ United States ┆ Ford      ┆ GT           ┆ 2017 ┆ … ┆ 18    ┆ rwd        ┆ 7a    ┆ 447000 │
│ Japan         ┆ Acura     ┆ NSX          ┆ 2017 ┆ … ┆ 22    ┆ awd        ┆ 9a    ┆ 156000 │
│ Japan         ┆ Nissan    ┆ GT-R         ┆ 2016 ┆ … ┆ 22    ┆ awd        ┆ 6a    ┆ 101770 │
└───────────────┴───────────┴──────────────┴──────┴───┴───────┴────────────┴───────┴────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33cc74940>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.stub: 2>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.default: 1>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='trim', column_align='left', column_width=None), ColInfo(var='bdy_style', type=<ColInfoTypeEnum.default: 1>, column_label='bdy_style', column_align='left', column_width=None), ColInfo(var='hp', type=<ColInfoTypeEnum.default: 1>, column_label='hp', column_align='center', column_width=None), ColInfo(var='hp_rpm', type=<ColInfoTypeEnum.default: 1>, column_label='hp_rpm', column_align='center', column_width=None), ColInfo(var='trq', type=<ColInfoTypeEnum.default: 1>, column_label='trq', column_align='center', column_width=None), ColInfo(var='trq_rpm', type=<ColInfoTypeEnum.default: 1>, column_label='trq_rpm', column_align='center', column_width=None), ColInfo(var='mpg_c', type=<ColInfoTypeEnum.default: 1>, column_label='mpg_c', column_align='center', column_width=None), ColInfo(var='mpg_h', type=<ColInfoTypeEnum.default: 1>, column_label='mpg_h', column_align='center', column_width=None), ColInfo(var='drivetrain', type=<ColInfoTypeEnum.default: 1>, column_label='drivetrain', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, column_label='trsmn', column_align='left', column_width=None), ColInfo(var='msrp', type=<ColInfoTypeEnum.default: 1>, column_label='msrp', column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id='Germany', rowname='BMW', group_label=None, built=False), RowInfo(rownum_i=1, group_id='Germany', rowname='BMW', group_label=None, built=False), RowInfo(rownum_i=2, group_id='Italy', rowname='Ferrari', group_label=None, built=False), RowInfo(rownum_i=3, group_id='Italy', rowname='Ferrari', group_label=None, built=False), RowInfo(rownum_i=4, group_id='United States', rowname='Chevrolet', group_label=None, built=False), RowInfo(rownum_i=5, group_id='United States', rowname='Ford', group_label=None, built=False), RowInfo(rownum_i=6, group_id='Japan', rowname='Acura', group_label=None, built=False), RowInfo(rownum_i=7, group_id='Japan', rowname='Nissan', group_label=None, built=False)]), _row_groups=['Germany', 

## Combined label

In [8]:
gtcars_8_combined_label = gtcars_8_pref_group_order.with_columns(
    c_label=pl.col("mfr").add(pl.lit(" ")).add(pl.col("model"))
)
gtcars_8_combined_label

ctry_origin,mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,msrp,c_label
str,str,str,u16,str,str,u32,u32,u32,u32,u8,u8,str,str,u32,str
"""Germany""","""BMW""","""i8""",2016,"""Mega World Coupe""","""coupe""",357,5800,420,3700,28,29,"""awd""","""6am""",140700,"""BMW i8"""
"""Germany""","""BMW""","""6-Series""",2016,"""640 I Coupe""","""coupe""",315,5800,330,1400,20,30,"""rwd""","""8am""",77300,"""BMW 6-Series"""
"""Italy""","""Ferrari""","""458 Speciale""",2015,"""Base Coupe""","""coupe""",597,9000,398,6000,13,17,"""rwd""","""7a""",291744,"""Ferrari 458 Speciale"""
"""Italy""","""Ferrari""","""458 Spider""",2015,"""Base""","""convertible""",562,9000,398,6000,13,17,"""rwd""","""7a""",263553,"""Ferrari 458 Spider"""
"""United States""","""Chevrolet""","""Corvette""",2016,"""Z06 Coupe""","""coupe""",650,6400,650,3600,15,22,"""rwd""","""7m""",88345,"""Chevrolet Corvette"""
"""United States""","""Ford""","""GT""",2017,"""Base Coupe""","""coupe""",647,6250,550,5900,11,18,"""rwd""","""7a""",447000,"""Ford GT"""
"""Japan""","""Acura""","""NSX""",2017,"""Base Coupe""","""coupe""",573,6500,476,2000,21,22,"""awd""","""9a""",156000,"""Acura NSX"""
"""Japan""","""Nissan""","""GT-R""",2016,"""Premium Coupe""","""coupe""",545,6400,436,3200,16,22,"""awd""","""6a""",101770,"""Nissan GT-R"""


In [9]:
(
    GT(
        gtcars_8_combined_label, groupname_col="ctry_origin", rowname_col="c_label"
    ).cols_hide(["mfr", "model"])
)

GT(_tbl_data=shape: (8, 16)
┌───────────────┬───────────┬──────────────┬──────┬───┬────────────┬───────┬────────┬──────────────┐
│ ctry_origin   ┆ mfr       ┆ model        ┆ year ┆ … ┆ drivetrain ┆ trsmn ┆ msrp   ┆ c_label      │
│ ---           ┆ ---       ┆ ---          ┆ ---  ┆   ┆ ---        ┆ ---   ┆ ---    ┆ ---          │
│ str           ┆ str       ┆ str          ┆ u16  ┆   ┆ str        ┆ str   ┆ u32    ┆ str          │
╞═══════════════╪═══════════╪══════════════╪══════╪═══╪════════════╪═══════╪════════╪══════════════╡
│ Germany       ┆ BMW       ┆ i8           ┆ 2016 ┆ … ┆ awd        ┆ 6am   ┆ 140700 ┆ BMW i8       │
│ Germany       ┆ BMW       ┆ 6-Series     ┆ 2016 ┆ … ┆ rwd        ┆ 8am   ┆ 77300  ┆ BMW 6-Series │
│ Italy         ┆ Ferrari   ┆ 458 Speciale ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 291744 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Speciale     │
│ Italy         ┆ Ferrari   ┆ 458 Spider   ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 263553 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Spider       │
│ United States ┆ Chevrolet ┆ Corvette     ┆ 2016 ┆ … ┆ rwd        ┆ 7m    ┆ 88345  ┆ Chevrolet    │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Corvette     │
│ United States ┆ Ford      ┆ GT           ┆ 2017 ┆ … ┆ rwd        ┆ 7a    ┆ 447000 ┆ Ford GT      │
│ Japan         ┆ Acura     ┆ NSX          ┆ 2017 ┆ … ┆ awd        ┆ 9a    ┆ 156000 ┆ Acura NSX    │
│ Japan         ┆ Nissan    ┆ GT-R         ┆ 2016 ┆ … ┆ awd        ┆ 6a    ┆ 101770 ┆ Nissan GT-R  │
└───────────────┴───────────┴──────────────┴──────┴───┴────────────┴───────┴────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33cc744f0>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='trim', column_align='left', column_width=None), ColInfo(var='bdy_style', type=<ColInfoTypeEnum.default: 1>, column_label='bdy_style', column_align='left', column_width=None), ColInfo(var='hp', type=<ColInfoTypeEnum.default: 1>, column_label='hp', column_align='center', column_width=None), ColInfo(var='hp_rpm', type=<ColInfoTypeEnum.default: 1>, column_label='hp_rpm', column_align='center', column_width=None), ColInfo(var='trq', type=<ColInfoTypeEnum.default: 1>, column_label='trq', column_align='center', column_width=None), ColInfo(var='trq_rpm', type=<ColInfoTypeEnum.default: 1>, column_label='trq_rpm', column_align='center', column_width=None), ColInfo(var='mpg_c', type=<ColInfoTypeEnum.default: 1>, column_label='mpg_c', column_align='center', column_width=None), ColInfo(var='mpg_h', type=<ColInfoTypeEnum.default: 1>, column_label='mpg_h', column_align='center', column_width=None), ColInfo(var='drivetrain', type=<ColInfoTypeEnum.default: 1>, column_label='drivetrain', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, column_label='trsmn', column_align='left', column_width=None), ColInfo(var='msrp', type=<ColInfoTypeEnum.default: 1>, column_label='msrp', column_align='center', column_width=None), ColInfo(var='c_label', type=<ColInfoTypeEnum.stub: 2>, column_label='c_label', column_align='left', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id='Germany', rowname='BMW i8', group_label=None, built=False), RowInfo(rownum_i=1, group_id='Germany', rowname='BMW 6-Series', group_label=None, built=False), RowInfo(rownum_i=2, group_id='Italy', 

## Hiding and Moving Some Columns

In [10]:
(
    GT(gtcars_8_combined_label, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(["mfr", "model", "drivetrain", "bdy_style"])
    .cols_move(["trsmn", "mpg_c", "mpg_h"], after="trim")
)

GT(_tbl_data=shape: (8, 16)
┌───────────────┬───────────┬──────────────┬──────┬───┬────────────┬───────┬────────┬──────────────┐
│ ctry_origin   ┆ mfr       ┆ model        ┆ year ┆ … ┆ drivetrain ┆ trsmn ┆ msrp   ┆ c_label      │
│ ---           ┆ ---       ┆ ---          ┆ ---  ┆   ┆ ---        ┆ ---   ┆ ---    ┆ ---          │
│ str           ┆ str       ┆ str          ┆ u16  ┆   ┆ str        ┆ str   ┆ u32    ┆ str          │
╞═══════════════╪═══════════╪══════════════╪══════╪═══╪════════════╪═══════╪════════╪══════════════╡
│ Germany       ┆ BMW       ┆ i8           ┆ 2016 ┆ … ┆ awd        ┆ 6am   ┆ 140700 ┆ BMW i8       │
│ Germany       ┆ BMW       ┆ 6-Series     ┆ 2016 ┆ … ┆ rwd        ┆ 8am   ┆ 77300  ┆ BMW 6-Series │
│ Italy         ┆ Ferrari   ┆ 458 Speciale ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 291744 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Speciale     │
│ Italy         ┆ Ferrari   ┆ 458 Spider   ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 263553 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Spider       │
│ United States ┆ Chevrolet ┆ Corvette     ┆ 2016 ┆ … ┆ rwd        ┆ 7m    ┆ 88345  ┆ Chevrolet    │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Corvette     │
│ United States ┆ Ford      ┆ GT           ┆ 2017 ┆ … ┆ rwd        ┆ 7a    ┆ 447000 ┆ Ford GT      │
│ Japan         ┆ Acura     ┆ NSX          ┆ 2017 ┆ … ┆ awd        ┆ 9a    ┆ 156000 ┆ Acura NSX    │
│ Japan         ┆ Nissan    ┆ GT-R         ┆ 2016 ┆ … ┆ awd        ┆ 6a    ┆ 101770 ┆ Nissan GT-R  │
└───────────────┴───────────┴──────────────┴──────┴───┴────────────┴───────┴────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb37d563f10>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='trim', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, column_label='trsmn', column_align='left', column_width=None), ColInfo(var='mpg_c', type=<ColInfoTypeEnum.default: 1>, column_label='mpg_c', column_align='center', column_width=None), ColInfo(var='mpg_h', type=<ColInfoTypeEnum.default: 1>, column_label='mpg_h', column_align='center', column_width=None), ColInfo(var='bdy_style', type=<ColInfoTypeEnum.hidden: 4>, column_label='bdy_style', column_align='left', column_width=None), ColInfo(var='hp', type=<ColInfoTypeEnum.default: 1>, column_label='hp', column_align='center', column_width=None), ColInfo(var='hp_rpm', type=<ColInfoTypeEnum.default: 1>, column_label='hp_rpm', column_align='center', column_width=None), ColInfo(var='trq', type=<ColInfoTypeEnum.default: 1>, column_label='trq', column_align='center', column_width=None), ColInfo(var='trq_rpm', type=<ColInfoTypeEnum.default: 1>, column_label='trq_rpm', column_align='center', column_width=None), ColInfo(var='drivetrain', type=<ColInfoTypeEnum.hidden: 4>, column_label='drivetrain', column_align='left', column_width=None), ColInfo(var='msrp', type=<ColInfoTypeEnum.default: 1>, column_label='msrp', column_align='center', column_width=None), ColInfo(var='c_label', type=<ColInfoTypeEnum.stub: 2>, column_label='c_label', column_align='left', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id='Germany', rowname='BMW i8', group_label=None, built=False), RowInfo(rownum_i=1, group_id='Germany', rowname='BMW 6-Series', group_label=None, built=False), RowInfo(rownum_i=2, group_id='Italy', ro

## Putting Columns Into Groups

In [11]:
(
    GT(gtcars_8_combined_label, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(["mfr", "model", "drivetrain", "bdy_style"])
    .cols_move(["trsmn", "mpg_c", "mpg_h"], after="trim")
    .tab_spanner(
        label="Performance",
        columns=["mpg_c", "mpg_h", "hp", "hp_rpm", "trq", "trq_rpm"],
    )
)

GT(_tbl_data=shape: (8, 16)
┌───────────────┬───────────┬──────────────┬──────┬───┬────────────┬───────┬────────┬──────────────┐
│ ctry_origin   ┆ mfr       ┆ model        ┆ year ┆ … ┆ drivetrain ┆ trsmn ┆ msrp   ┆ c_label      │
│ ---           ┆ ---       ┆ ---          ┆ ---  ┆   ┆ ---        ┆ ---   ┆ ---    ┆ ---          │
│ str           ┆ str       ┆ str          ┆ u16  ┆   ┆ str        ┆ str   ┆ u32    ┆ str          │
╞═══════════════╪═══════════╪══════════════╪══════╪═══╪════════════╪═══════╪════════╪══════════════╡
│ Germany       ┆ BMW       ┆ i8           ┆ 2016 ┆ … ┆ awd        ┆ 6am   ┆ 140700 ┆ BMW i8       │
│ Germany       ┆ BMW       ┆ 6-Series     ┆ 2016 ┆ … ┆ rwd        ┆ 8am   ┆ 77300  ┆ BMW 6-Series │
│ Italy         ┆ Ferrari   ┆ 458 Speciale ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 291744 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Speciale     │
│ Italy         ┆ Ferrari   ┆ 458 Spider   ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 263553 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Spider       │
│ United States ┆ Chevrolet ┆ Corvette     ┆ 2016 ┆ … ┆ rwd        ┆ 7m    ┆ 88345  ┆ Chevrolet    │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Corvette     │
│ United States ┆ Ford      ┆ GT           ┆ 2017 ┆ … ┆ rwd        ┆ 7a    ┆ 447000 ┆ Ford GT      │
│ Japan         ┆ Acura     ┆ NSX          ┆ 2017 ┆ … ┆ awd        ┆ 9a    ┆ 156000 ┆ Acura NSX    │
│ Japan         ┆ Nissan    ┆ GT-R         ┆ 2016 ┆ … ┆ awd        ┆ 6a    ┆ 101770 ┆ Nissan GT-R  │
└───────────────┴───────────┴──────────────┴──────┴───┴────────────┴───────┴────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33e726c80>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='trim', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, column_label='trsmn', column_align='left', column_width=None), ColInfo(var='mpg_c', type=<ColInfoTypeEnum.default: 1>, column_label='mpg_c', column_align='center', column_width=None), ColInfo(var='mpg_h', type=<ColInfoTypeEnum.default: 1>, column_label='mpg_h', column_align='center', column_width=None), ColInfo(var='hp', type=<ColInfoTypeEnum.default: 1>, column_label='hp', column_align='center', column_width=None), ColInfo(var='hp_rpm', type=<ColInfoTypeEnum.default: 1>, column_label='hp_rpm', column_align='center', column_width=None), ColInfo(var='trq', type=<ColInfoTypeEnum.default: 1>, column_label='trq', column_align='center', column_width=None), ColInfo(var='trq_rpm', type=<ColInfoTypeEnum.default: 1>, column_label='trq_rpm', column_align='center', column_width=None), ColInfo(var='bdy_style', type=<ColInfoTypeEnum.hidden: 4>, column_label='bdy_style', column_align='left', column_width=None), ColInfo(var='drivetrain', type=<ColInfoTypeEnum.hidden: 4>, column_label='drivetrain', column_align='left', column_width=None), ColInfo(var='msrp', type=<ColInfoTypeEnum.default: 1>, column_label='msrp', column_align='center', column_width=None), ColInfo(var='c_label', type=<ColInfoTypeEnum.stub: 2>, column_label='c_label', column_align='left', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id='Germany', rowname='BMW i8', group_label=None, built=False), RowInfo(rownum_i=1, group_id='Germany', rowname='BMW 6-Series', group_label=None, built=False), RowInfo(rownum_i=2, group_id='Italy', ro

## Merging Columns Together and Labeling Them

In [12]:
def fmt_performance(col1: str, c1: str, col2: str, c2: str) -> pl.Expr:
    return (
        pl.col(col1)
        .cast(pl.Utf8)
        .add(pl.lit(c1))
        .add(pl.col(col2).cast(pl.Utf8).add(pl.lit(c2)))
    )


gtcars_8_merge_cols = gtcars_8_combined_label.with_columns(
    mpg_c=fmt_performance("mpg_c", "c<br>", "mpg_h", "h"),
    hp=fmt_performance("hp", "<br>@", "hp_rpm", "rpm"),
    trq=fmt_performance("trq", "<br>@", "trq_rpm", "rpm"),
)
gtcars_8_merge_cols

ctry_origin,mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,msrp,c_label
str,str,str,u16,str,str,str,u32,str,u32,str,u8,str,str,u32,str
"""Germany""","""BMW""","""i8""",2016,"""Mega World Coupe""","""coupe""","""357<br>@5800rpm""",5800,"""420<br>@3700rpm""",3700,"""28c<br>29h""",29,"""awd""","""6am""",140700,"""BMW i8"""
"""Germany""","""BMW""","""6-Series""",2016,"""640 I Coupe""","""coupe""","""315<br>@5800rpm""",5800,"""330<br>@1400rpm""",1400,"""20c<br>30h""",30,"""rwd""","""8am""",77300,"""BMW 6-Series"""
"""Italy""","""Ferrari""","""458 Speciale""",2015,"""Base Coupe""","""coupe""","""597<br>@9000rpm""",9000,"""398<br>@6000rpm""",6000,"""13c<br>17h""",17,"""rwd""","""7a""",291744,"""Ferrari 458 Speciale"""
"""Italy""","""Ferrari""","""458 Spider""",2015,"""Base""","""convertible""","""562<br>@9000rpm""",9000,"""398<br>@6000rpm""",6000,"""13c<br>17h""",17,"""rwd""","""7a""",263553,"""Ferrari 458 Spider"""
"""United States""","""Chevrolet""","""Corvette""",2016,"""Z06 Coupe""","""coupe""","""650<br>@6400rpm""",6400,"""650<br>@3600rpm""",3600,"""15c<br>22h""",22,"""rwd""","""7m""",88345,"""Chevrolet Corvette"""
"""United States""","""Ford""","""GT""",2017,"""Base Coupe""","""coupe""","""647<br>@6250rpm""",6250,"""550<br>@5900rpm""",5900,"""11c<br>18h""",18,"""rwd""","""7a""",447000,"""Ford GT"""
"""Japan""","""Acura""","""NSX""",2017,"""Base Coupe""","""coupe""","""573<br>@6500rpm""",6500,"""476<br>@2000rpm""",2000,"""21c<br>22h""",22,"""awd""","""9a""",156000,"""Acura NSX"""
"""Japan""","""Nissan""","""GT-R""",2016,"""Premium Coupe""","""coupe""","""545<br>@6400rpm""",6400,"""436<br>@3200rpm""",3200,"""16c<br>22h""",22,"""awd""","""6a""",101770,"""Nissan GT-R"""


In [13]:
col_labels = {
    "mpg_c": "MPG",
    "hp": "HP",
    "trq": "Torque",
    "year": "Year",
    "trim": "Trim",
    "trsmn": "Transmission",
    "msrp": "MSRP",
}

(
    GT(gtcars_8_merge_cols, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(
        ["mfr", "model", "drivetrain", "bdy_style", "mpg_h", "hp_rpm", "trq_rpm"]
    )
    .cols_move(["trsmn", "mpg_c"], after="trim")
    .tab_spanner(label="Performance", columns=["mpg_c", "hp", "trq"])
    .cols_label(**col_labels)
)

GT(_tbl_data=shape: (8, 16)
┌───────────────┬───────────┬──────────────┬──────┬───┬────────────┬───────┬────────┬──────────────┐
│ ctry_origin   ┆ mfr       ┆ model        ┆ year ┆ … ┆ drivetrain ┆ trsmn ┆ msrp   ┆ c_label      │
│ ---           ┆ ---       ┆ ---          ┆ ---  ┆   ┆ ---        ┆ ---   ┆ ---    ┆ ---          │
│ str           ┆ str       ┆ str          ┆ u16  ┆   ┆ str        ┆ str   ┆ u32    ┆ str          │
╞═══════════════╪═══════════╪══════════════╪══════╪═══╪════════════╪═══════╪════════╪══════════════╡
│ Germany       ┆ BMW       ┆ i8           ┆ 2016 ┆ … ┆ awd        ┆ 6am   ┆ 140700 ┆ BMW i8       │
│ Germany       ┆ BMW       ┆ 6-Series     ┆ 2016 ┆ … ┆ rwd        ┆ 8am   ┆ 77300  ┆ BMW 6-Series │
│ Italy         ┆ Ferrari   ┆ 458 Speciale ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 291744 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Speciale     │
│ Italy         ┆ Ferrari   ┆ 458 Spider   ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 263553 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Spider       │
│ United States ┆ Chevrolet ┆ Corvette     ┆ 2016 ┆ … ┆ rwd        ┆ 7m    ┆ 88345  ┆ Chevrolet    │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Corvette     │
│ United States ┆ Ford      ┆ GT           ┆ 2017 ┆ … ┆ rwd        ┆ 7a    ┆ 447000 ┆ Ford GT      │
│ Japan         ┆ Acura     ┆ NSX          ┆ 2017 ┆ … ┆ awd        ┆ 9a    ┆ 156000 ┆ Acura NSX    │
│ Japan         ┆ Nissan    ┆ GT-R         ┆ 2016 ┆ … ┆ awd        ┆ 6a    ┆ 101770 ┆ Nissan GT-R  │
└───────────────┴───────────┴──────────────┴──────┴───┴────────────┴───────┴────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33e2bfe50>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='Trim', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, column_label='Transmission', column_align='left', column_width=None), ColInfo(var='mpg_c', type=<ColInfoTypeEnum.default: 1>, column_label='MPG', column_align='left', column_width=None), ColInfo(var='hp', type=<ColInfoTypeEnum.default: 1>, column_label='HP', column_align='left', column_width=None), ColInfo(var='trq', type=<ColInfoTypeEnum.default: 1>, column_label='Torque', column_align='left', column_width=None), ColInfo(var='bdy_style', type=<ColInfoTypeEnum.hidden: 4>, column_label='bdy_style', column_align='left', column_width=None), ColInfo(var='hp_rpm', type=<ColInfoTypeEnum.hidden: 4>, column_label='hp_rpm', column_align='center', column_width=None), ColInfo(var='trq_rpm', type=<ColInfoTypeEnum.hidden: 4>, column_label='trq_rpm', column_align='center', column_width=None), ColInfo(var='mpg_h', type=<ColInfoTypeEnum.hidden: 4>, column_label='mpg_h', column_align='center', column_width=None), ColInfo(var='drivetrain', type=<ColInfoTypeEnum.hidden: 4>, column_label='drivetrain', column_align='left', column_width=None), ColInfo(var='msrp', type=<ColInfoTypeEnum.default: 1>, column_label='MSRP', column_align='center', column_width=None), ColInfo(var='c_label', type=<ColInfoTypeEnum.stub: 2>, column_label='c_label', column_align='left', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id='Germany', rowname='BMW i8', group_label=None, built=False), RowInfo(rownum_i=1, group_id='Germany', rowname='BMW 6-Series', group_label=None, built=False), RowInfo(rownum_i=2, group_id='Italy', row

## Using Formatter Functions

In [14]:
(
    GT(gtcars_8_merge_cols, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(
        ["mfr", "model", "drivetrain", "bdy_style", "mpg_h", "hp_rpm", "trq_rpm"]
    )
    .cols_move(["trsmn", "mpg_c"], after="trim")
    .tab_spanner(label="Performance", columns=["mpg_c", "hp", "trq"])
    .cols_label(**col_labels)
    .fmt_currency("msrp", decimals=0)
)

GT(_tbl_data=shape: (8, 16)
┌───────────────┬───────────┬──────────────┬──────┬───┬────────────┬───────┬────────┬──────────────┐
│ ctry_origin   ┆ mfr       ┆ model        ┆ year ┆ … ┆ drivetrain ┆ trsmn ┆ msrp   ┆ c_label      │
│ ---           ┆ ---       ┆ ---          ┆ ---  ┆   ┆ ---        ┆ ---   ┆ ---    ┆ ---          │
│ str           ┆ str       ┆ str          ┆ u16  ┆   ┆ str        ┆ str   ┆ u32    ┆ str          │
╞═══════════════╪═══════════╪══════════════╪══════╪═══╪════════════╪═══════╪════════╪══════════════╡
│ Germany       ┆ BMW       ┆ i8           ┆ 2016 ┆ … ┆ awd        ┆ 6am   ┆ 140700 ┆ BMW i8       │
│ Germany       ┆ BMW       ┆ 6-Series     ┆ 2016 ┆ … ┆ rwd        ┆ 8am   ┆ 77300  ┆ BMW 6-Series │
│ Italy         ┆ Ferrari   ┆ 458 Speciale ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 291744 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Speciale     │
│ Italy         ┆ Ferrari   ┆ 458 Spider   ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 263553 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Spider       │
│ United States ┆ Chevrolet ┆ Corvette     ┆ 2016 ┆ … ┆ rwd        ┆ 7m    ┆ 88345  ┆ Chevrolet    │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Corvette     │
│ United States ┆ Ford      ┆ GT           ┆ 2017 ┆ … ┆ rwd        ┆ 7a    ┆ 447000 ┆ Ford GT      │
│ Japan         ┆ Acura     ┆ NSX          ┆ 2017 ┆ … ┆ awd        ┆ 9a    ┆ 156000 ┆ Acura NSX    │
│ Japan         ┆ Nissan    ┆ GT-R         ┆ 2016 ┆ … ┆ awd        ┆ 6a    ┆ 101770 ┆ Nissan GT-R  │
└───────────────┴───────────┴──────────────┴──────┴───┴────────────┴───────┴────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33e2bf8b0>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='Trim', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, column_label='Transmission', column_align='left', column_width=None), ColInfo(var='mpg_c', type=<ColInfoTypeEnum.default: 1>, column_label='MPG', column_align='left', column_width=None), ColInfo(var='hp', type=<ColInfoTypeEnum.default: 1>, column_label='HP', column_align='left', column_width=None), ColInfo(var='trq', type=<ColInfoTypeEnum.default: 1>, column_label='Torque', column_align='left', column_width=None), ColInfo(var='bdy_style', type=<ColInfoTypeEnum.hidden: 4>, column_label='bdy_style', column_align='left', column_width=None), ColInfo(var='hp_rpm', type=<ColInfoTypeEnum.hidden: 4>, column_label='hp_rpm', column_align='center', column_width=None), ColInfo(var='trq_rpm', type=<ColInfoTypeEnum.hidden: 4>, column_label='trq_rpm', column_align='center', column_width=None), ColInfo(var='mpg_h', type=<ColInfoTypeEnum.hidden: 4>, column_label='mpg_h', column_align='center', column_width=None), ColInfo(var='drivetrain', type=<ColInfoTypeEnum.hidden: 4>, column_label='drivetrain', column_align='left', column_width=None), ColInfo(var='msrp', type=<ColInfoTypeEnum.default: 1>, column_label='MSRP', column_align='center', column_width=None), ColInfo(var='c_label', type=<ColInfoTypeEnum.stub: 2>, column_label='c_label', column_align='left', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id='Germany', rowname='BMW i8', group_label=None, built=False), RowInfo(rownum_i=1, group_id='Germany', rowname='BMW 6-Series', group_label=None, built=False), RowInfo(rownum_i=2, group_id='Italy', row

## Column Alignment and Style Changes

In [15]:
(
    GT(gtcars_8_merge_cols, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(
        ["mfr", "model", "drivetrain", "bdy_style", "mpg_h", "hp_rpm", "trq_rpm"]
    )
    .cols_move(["trsmn", "mpg_c"], after="trim")
    .tab_spanner(label="Performance", columns=["mpg_c", "hp", "trq"])
    .cols_label(**col_labels)
    .fmt_currency("msrp", decimals=0)
    .cols_align(align="center", columns=["mpg_c", "hp", "trq"])
    .tab_style(
        style=style.text(size=px("12")),
        locations=loc.body(columns=["trim", "trsmn", "mpg_c", "hp", "trq"]),
    )
)

GT(_tbl_data=shape: (8, 16)
┌───────────────┬───────────┬──────────────┬──────┬───┬────────────┬───────┬────────┬──────────────┐
│ ctry_origin   ┆ mfr       ┆ model        ┆ year ┆ … ┆ drivetrain ┆ trsmn ┆ msrp   ┆ c_label      │
│ ---           ┆ ---       ┆ ---          ┆ ---  ┆   ┆ ---        ┆ ---   ┆ ---    ┆ ---          │
│ str           ┆ str       ┆ str          ┆ u16  ┆   ┆ str        ┆ str   ┆ u32    ┆ str          │
╞═══════════════╪═══════════╪══════════════╪══════╪═══╪════════════╪═══════╪════════╪══════════════╡
│ Germany       ┆ BMW       ┆ i8           ┆ 2016 ┆ … ┆ awd        ┆ 6am   ┆ 140700 ┆ BMW i8       │
│ Germany       ┆ BMW       ┆ 6-Series     ┆ 2016 ┆ … ┆ rwd        ┆ 8am   ┆ 77300  ┆ BMW 6-Series │
│ Italy         ┆ Ferrari   ┆ 458 Speciale ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 291744 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Speciale     │
│ Italy         ┆ Ferrari   ┆ 458 Spider   ┆ 2015 ┆ … ┆ rwd        ┆ 7a    ┆ 263553 ┆ Ferrari 458  │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Spider       │
│ United States ┆ Chevrolet ┆ Corvette     ┆ 2016 ┆ … ┆ rwd        ┆ 7m    ┆ 88345  ┆ Chevrolet    │
│               ┆           ┆              ┆      ┆   ┆            ┆       ┆        ┆ Corvette     │
│ United States ┆ Ford      ┆ GT           ┆ 2017 ┆ … ┆ rwd        ┆ 7a    ┆ 447000 ┆ Ford GT      │
│ Japan         ┆ Acura     ┆ NSX          ┆ 2017 ┆ … ┆ awd        ┆ 9a    ┆ 156000 ┆ Acura NSX    │
│ Japan         ┆ Nissan    ┆ GT-R         ┆ 2016 ┆ … ┆ awd        ┆ 6a    ┆ 101770 ┆ Nissan GT-R  │
└───────────────┴───────────┴──────────────┴──────┴───┴────────────┴───────┴────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33cc74760>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='Trim', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, column_label='Transmission', column_align='left', column_width=None), ColInfo(var='mpg_c', type=<ColInfoTypeEnum.default: 1>, column_label='MPG', column_align='center', column_width=None), ColInfo(var='hp', type=<ColInfoTypeEnum.default: 1>, column_label='HP', column_align='center', column_width=None), ColInfo(var='trq', type=<ColInfoTypeEnum.default: 1>, column_label='Torque', column_align='center', column_width=None), ColInfo(var='bdy_style', type=<ColInfoTypeEnum.hidden: 4>, column_label='bdy_style', column_align='left', column_width=None), ColInfo(var='hp_rpm', type=<ColInfoTypeEnum.hidden: 4>, column_label='hp_rpm', column_align='center', column_width=None), ColInfo(var='trq_rpm', type=<ColInfoTypeEnum.hidden: 4>, column_label='trq_rpm', column_align='center', column_width=None), ColInfo(var='mpg_h', type=<ColInfoTypeEnum.hidden: 4>, column_label='mpg_h', column_align='center', column_width=None), ColInfo(var='drivetrain', type=<ColInfoTypeEnum.hidden: 4>, column_label='drivetrain', column_align='left', column_width=None), ColInfo(var='msrp', type=<ColInfoTypeEnum.default: 1>, column_label='MSRP', column_align='center', column_width=None), ColInfo(var='c_label', type=<ColInfoTypeEnum.stub: 2>, column_label='c_label', column_align='left', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id='Germany', rowname='BMW i8', group_label=None, built=False), RowInfo(rownum_i=1, group_id='Germany', rowname='BMW 6-Series', group_label=None, built=False), RowInfo(rownum_i=2, group_id='Italy

## Text Transforms

In [16]:
gtcars_8_text_transform = (
    gtcars_8_merge_cols.with_columns(
        speed=pl.col("trsmn").str.slice(0, 1).cast(pl.UInt8),
        type_=pl.col("trsmn").str.slice(1),
    )
    .with_columns(
        type_=pl.when(pl.col("type_").eq("am"))
        .then(pl.lit("Automatic/Manual"))
        .when(pl.col("type_").eq("m"))
        .then(pl.lit("Manual"))
        .when(pl.col("type_").eq("a"))
        .then(pl.lit("Automatic"))
        .otherwise(pl.lit("Direct Drive"))
    )
    .with_columns(
        trsmn=pl.col("speed")
        .cast(pl.Utf8)
        .add(pl.lit(" Speed<br><em>").add(pl.col("type_")))
        .add(pl.lit("</em>"))
    )
    .select(pl.all().exclude(["speed", "type_"]))
)
gtcars_8_text_transform

ctry_origin,mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,msrp,c_label
str,str,str,u16,str,str,str,u32,str,u32,str,u8,str,str,u32,str
"""Germany""","""BMW""","""i8""",2016,"""Mega World Coupe""","""coupe""","""357<br>@5800rpm""",5800,"""420<br>@3700rpm""",3700,"""28c<br>29h""",29,"""awd""","""6 Speed<br><em>Automatic/Manua…",140700,"""BMW i8"""
"""Germany""","""BMW""","""6-Series""",2016,"""640 I Coupe""","""coupe""","""315<br>@5800rpm""",5800,"""330<br>@1400rpm""",1400,"""20c<br>30h""",30,"""rwd""","""8 Speed<br><em>Automatic/Manua…",77300,"""BMW 6-Series"""
"""Italy""","""Ferrari""","""458 Speciale""",2015,"""Base Coupe""","""coupe""","""597<br>@9000rpm""",9000,"""398<br>@6000rpm""",6000,"""13c<br>17h""",17,"""rwd""","""7 Speed<br><em>Automatic</em>""",291744,"""Ferrari 458 Speciale"""
"""Italy""","""Ferrari""","""458 Spider""",2015,"""Base""","""convertible""","""562<br>@9000rpm""",9000,"""398<br>@6000rpm""",6000,"""13c<br>17h""",17,"""rwd""","""7 Speed<br><em>Automatic</em>""",263553,"""Ferrari 458 Spider"""
"""United States""","""Chevrolet""","""Corvette""",2016,"""Z06 Coupe""","""coupe""","""650<br>@6400rpm""",6400,"""650<br>@3600rpm""",3600,"""15c<br>22h""",22,"""rwd""","""7 Speed<br><em>Manual</em>""",88345,"""Chevrolet Corvette"""
"""United States""","""Ford""","""GT""",2017,"""Base Coupe""","""coupe""","""647<br>@6250rpm""",6250,"""550<br>@5900rpm""",5900,"""11c<br>18h""",18,"""rwd""","""7 Speed<br><em>Automatic</em>""",447000,"""Ford GT"""
"""Japan""","""Acura""","""NSX""",2017,"""Base Coupe""","""coupe""","""573<br>@6500rpm""",6500,"""476<br>@2000rpm""",2000,"""21c<br>22h""",22,"""awd""","""9 Speed<br><em>Automatic</em>""",156000,"""Acura NSX"""
"""Japan""","""Nissan""","""GT-R""",2016,"""Premium Coupe""","""coupe""","""545<br>@6400rpm""",6400,"""436<br>@3200rpm""",3200,"""16c<br>22h""",22,"""awd""","""6 Speed<br><em>Automatic</em>""",101770,"""Nissan GT-R"""


In [17]:
(
    GT(gtcars_8_text_transform, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(
        ["mfr", "model", "drivetrain", "bdy_style", "mpg_h", "hp_rpm", "trq_rpm"]
    )
    .cols_move(["trsmn", "mpg_c"], after="trim")
    .tab_spanner(label="Performance", columns=["mpg_c", "hp", "trq"])
    .cols_label(**col_labels)
    .fmt_currency("msrp", decimals=0)
    .cols_align(align="center", columns=["mpg_c", "hp", "trq"])
    .tab_style(
        style=style.text(size=px("12")),
        locations=loc.body(columns=["trim", "trsmn", "mpg_c", "hp", "trq"]),
    )
)

GT(_tbl_data=shape: (8, 16)
┌─────────────┬───────────┬─────────────┬──────┬───┬────────────┬────────────┬────────┬────────────┐
│ ctry_origin ┆ mfr       ┆ model       ┆ year ┆ … ┆ drivetrain ┆ trsmn      ┆ msrp   ┆ c_label    │
│ ---         ┆ ---       ┆ ---         ┆ ---  ┆   ┆ ---        ┆ ---        ┆ ---    ┆ ---        │
│ str         ┆ str       ┆ str         ┆ u16  ┆   ┆ str        ┆ str        ┆ u32    ┆ str        │
╞═════════════╪═══════════╪═════════════╪══════╪═══╪════════════╪════════════╪════════╪════════════╡
│ Germany     ┆ BMW       ┆ i8          ┆ 2016 ┆ … ┆ awd        ┆ 6 Speed<br ┆ 140700 ┆ BMW i8     │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic/Manua ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ …          ┆        ┆            │
│ Germany     ┆ BMW       ┆ 6-Series    ┆ 2016 ┆ … ┆ rwd        ┆ 8 Speed<br ┆ 77300  ┆ BMW        │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 6-Series   │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic/Manua ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ …          ┆        ┆            │
│ Italy       ┆ Ferrari   ┆ 458         ┆ 2015 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 291744 ┆ Ferrari    │
│             ┆           ┆ Speciale    ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 458        │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆ Speciale   │
│ Italy       ┆ Ferrari   ┆ 458 Spider  ┆ 2015 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 263553 ┆ Ferrari    │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 458 Spider │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ United      ┆ Chevrolet ┆ Corvette    ┆ 2016 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 88345  ┆ Chevrolet  │
│ States      ┆           ┆             ┆      ┆   ┆            ┆ ><em>Manua ┆        ┆ Corvette   │
│             ┆           ┆             ┆      ┆   ┆            ┆ l</em>     ┆        ┆            │
│ United      ┆ Ford      ┆ GT          ┆ 2017 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 447000 ┆ Ford GT    │
│ States      ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ Japan       ┆ Acura     ┆ NSX         ┆ 2017 ┆ … ┆ awd        ┆ 9 Speed<br ┆ 156000 ┆ Acura NSX  │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ Japan       ┆ Nissan    ┆ GT-R        ┆ 2016 ┆ … ┆ awd        ┆ 6 Speed<br ┆ 101770 ┆ Nissan     │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ GT-R       │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
└─────────────┴───────────┴─────────────┴──────┴───┴────────────┴────────────┴────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33e2a4070>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='Trim', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, col

## Table Header: Title and Subtitle

In [18]:
(
    GT(gtcars_8_text_transform, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(
        ["mfr", "model", "drivetrain", "bdy_style", "mpg_h", "hp_rpm", "trq_rpm"]
    )
    .cols_move(["trsmn", "mpg_c"], after="trim")
    .tab_spanner(label="Performance", columns=["mpg_c", "hp", "trq"])
    .cols_label(**col_labels)
    .fmt_currency("msrp", decimals=0)
    .cols_align(align="center", columns=["mpg_c", "hp", "trq"])
    .tab_style(
        style=style.text(size=px("12")),
        locations=loc.body(columns=["trim", "trsmn", "mpg_c", "hp", "trq"]),
    )
    .tab_header(
        title=md("The Cars of **gtcars**"), subtitle="These are some fine automobiles"
    )
)

GT(_tbl_data=shape: (8, 16)
┌─────────────┬───────────┬─────────────┬──────┬───┬────────────┬────────────┬────────┬────────────┐
│ ctry_origin ┆ mfr       ┆ model       ┆ year ┆ … ┆ drivetrain ┆ trsmn      ┆ msrp   ┆ c_label    │
│ ---         ┆ ---       ┆ ---         ┆ ---  ┆   ┆ ---        ┆ ---        ┆ ---    ┆ ---        │
│ str         ┆ str       ┆ str         ┆ u16  ┆   ┆ str        ┆ str        ┆ u32    ┆ str        │
╞═════════════╪═══════════╪═════════════╪══════╪═══╪════════════╪════════════╪════════╪════════════╡
│ Germany     ┆ BMW       ┆ i8          ┆ 2016 ┆ … ┆ awd        ┆ 6 Speed<br ┆ 140700 ┆ BMW i8     │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic/Manua ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ …          ┆        ┆            │
│ Germany     ┆ BMW       ┆ 6-Series    ┆ 2016 ┆ … ┆ rwd        ┆ 8 Speed<br ┆ 77300  ┆ BMW        │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 6-Series   │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic/Manua ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ …          ┆        ┆            │
│ Italy       ┆ Ferrari   ┆ 458         ┆ 2015 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 291744 ┆ Ferrari    │
│             ┆           ┆ Speciale    ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 458        │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆ Speciale   │
│ Italy       ┆ Ferrari   ┆ 458 Spider  ┆ 2015 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 263553 ┆ Ferrari    │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 458 Spider │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ United      ┆ Chevrolet ┆ Corvette    ┆ 2016 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 88345  ┆ Chevrolet  │
│ States      ┆           ┆             ┆      ┆   ┆            ┆ ><em>Manua ┆        ┆ Corvette   │
│             ┆           ┆             ┆      ┆   ┆            ┆ l</em>     ┆        ┆            │
│ United      ┆ Ford      ┆ GT          ┆ 2017 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 447000 ┆ Ford GT    │
│ States      ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ Japan       ┆ Acura     ┆ NSX         ┆ 2017 ┆ … ┆ awd        ┆ 9 Speed<br ┆ 156000 ┆ Acura NSX  │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ Japan       ┆ Nissan    ┆ GT-R        ┆ 2016 ┆ … ┆ awd        ┆ 6 Speed<br ┆ 101770 ┆ Nissan     │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ GT-R       │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
└─────────────┴───────────┴─────────────┴──────┴───┴────────────┴────────────┴────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33e287280>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='Trim', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, col

## Adding a Source Citation

In [19]:
(
    GT(gtcars_8_text_transform, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(
        ["mfr", "model", "drivetrain", "bdy_style", "mpg_h", "hp_rpm", "trq_rpm"]
    )
    .cols_move(["trsmn", "mpg_c"], after="trim")
    .tab_spanner(label="Performance", columns=["mpg_c", "hp", "trq"])
    .cols_label(**col_labels)
    .fmt_currency("msrp", decimals=0)
    .cols_align(align="center", columns=["mpg_c", "hp", "trq"])
    .tab_style(
        style=style.text(size=px("12")),
        locations=loc.body(columns=["trim", "trsmn", "mpg_c", "hp", "trq"]),
    )
    .tab_header(
        title=md("The Cars of **gtcars**"), subtitle="These are some fine automobiles"
    )
    .tab_source_note(md("Source: Various pages within the Edmonds website."))
)

GT(_tbl_data=shape: (8, 16)
┌─────────────┬───────────┬─────────────┬──────┬───┬────────────┬────────────┬────────┬────────────┐
│ ctry_origin ┆ mfr       ┆ model       ┆ year ┆ … ┆ drivetrain ┆ trsmn      ┆ msrp   ┆ c_label    │
│ ---         ┆ ---       ┆ ---         ┆ ---  ┆   ┆ ---        ┆ ---        ┆ ---    ┆ ---        │
│ str         ┆ str       ┆ str         ┆ u16  ┆   ┆ str        ┆ str        ┆ u32    ┆ str        │
╞═════════════╪═══════════╪═════════════╪══════╪═══╪════════════╪════════════╪════════╪════════════╡
│ Germany     ┆ BMW       ┆ i8          ┆ 2016 ┆ … ┆ awd        ┆ 6 Speed<br ┆ 140700 ┆ BMW i8     │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic/Manua ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ …          ┆        ┆            │
│ Germany     ┆ BMW       ┆ 6-Series    ┆ 2016 ┆ … ┆ rwd        ┆ 8 Speed<br ┆ 77300  ┆ BMW        │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 6-Series   │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic/Manua ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ …          ┆        ┆            │
│ Italy       ┆ Ferrari   ┆ 458         ┆ 2015 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 291744 ┆ Ferrari    │
│             ┆           ┆ Speciale    ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 458        │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆ Speciale   │
│ Italy       ┆ Ferrari   ┆ 458 Spider  ┆ 2015 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 263553 ┆ Ferrari    │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ 458 Spider │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ United      ┆ Chevrolet ┆ Corvette    ┆ 2016 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 88345  ┆ Chevrolet  │
│ States      ┆           ┆             ┆      ┆   ┆            ┆ ><em>Manua ┆        ┆ Corvette   │
│             ┆           ┆             ┆      ┆   ┆            ┆ l</em>     ┆        ┆            │
│ United      ┆ Ford      ┆ GT          ┆ 2017 ┆ … ┆ rwd        ┆ 7 Speed<br ┆ 447000 ┆ Ford GT    │
│ States      ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ Japan       ┆ Acura     ┆ NSX         ┆ 2017 ┆ … ┆ awd        ┆ 9 Speed<br ┆ 156000 ┆ Acura NSX  │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆            │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
│ Japan       ┆ Nissan    ┆ GT-R        ┆ 2016 ┆ … ┆ awd        ┆ 6 Speed<br ┆ 101770 ┆ Nissan     │
│             ┆           ┆             ┆      ┆   ┆            ┆ ><em>Autom ┆        ┆ GT-R       │
│             ┆           ┆             ┆      ┆   ┆            ┆ atic</em>  ┆        ┆            │
└─────────────┴───────────┴─────────────┴──────┴───┴────────────┴────────────┴────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33e2a4c40>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', type=<ColInfoTypeEnum.hidden: 4>, column_label='mfr', column_align='left', column_width=None), ColInfo(var='model', type=<ColInfoTypeEnum.hidden: 4>, column_label='model', column_align='left', column_width=None), ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='center', column_width=None), ColInfo(var='trim', type=<ColInfoTypeEnum.default: 1>, column_label='Trim', column_align='left', column_width=None), ColInfo(var='trsmn', type=<ColInfoTypeEnum.default: 1>, col

## Using the Complete gtcars table

In [20]:
gtcars_final = (
    gtcars.sort(pl.col("ctry_origin").replace(order_countries, return_dtype=pl.UInt8))
    .select(pl.col("ctry_origin"), pl.all().exclude("ctry_origin"))
    .select(
        pl.all().sort_by(["mfr", "msrp"], descending=[False, True]).over("ctry_origin")
    )
    .with_columns(c_label=pl.col("mfr").add(pl.lit(" ")).add(pl.col("model")))
    .with_columns(
        mpg_c=fmt_performance("mpg_c", "c<br>", "mpg_h", "h"),
        hp=fmt_performance("hp", "<br>@", "hp_rpm", "rpm"),
        trq=fmt_performance("trq", "<br>@", "trq_rpm", "rpm"),
    )
    .with_columns(
        speed=pl.col("trsmn").str.slice(0, 1).cast(pl.UInt8),
        type_=pl.col("trsmn").str.slice(1),
    )
    .with_columns(
        type_=pl.when(pl.col("type_").eq("am"))
        .then(pl.lit("Automatic/Manual"))
        .when(pl.col("type_").eq("m"))
        .then(pl.lit("Manual"))
        .when(pl.col("type_").eq("a"))
        .then(pl.lit("Automatic"))
        .otherwise(pl.lit("Direct Drive"))
    )
    .with_columns(
        trsmn=pl.col("speed")
        .cast(pl.Utf8)
        .add(pl.lit(" Speed<br><em>").add(pl.col("type_")))
        .add(pl.lit("</em>"))
    )
    .select(pl.all().exclude(["speed", "type_"]))
)
gtcars_final.head()

ctry_origin,mfr,model,year,trim,bdy_style,hp,hp_rpm,trq,trq_rpm,mpg_c,mpg_h,drivetrain,trsmn,msrp,c_label
str,str,str,u16,str,str,str,u32,str,u32,str,u8,str,str,u32,str
"""Germany""","""Audi""","""R8""",2015,"""4.2 (Manual) Coupe""","""coupe""","""430<br>@7900rpm""",7900,"""317<br>@4500rpm""",4500,"""11c<br>20h""",20,"""awd""","""6 Speed<br><em>Manual</em>""",115900,"""Audi R8"""
"""Germany""","""Audi""","""S8""",2016,"""Base Sedan""","""sedan""","""520<br>@5800rpm""",5800,"""481<br>@1700rpm""",1700,"""15c<br>25h""",25,"""awd""","""8 Speed<br><em>Automatic/Manua…",114900,"""Audi S8"""
"""Germany""","""Audi""","""RS 7""",2016,"""Quattro Hatchback""","""hatchback""","""560<br>@5700rpm""",5700,"""516<br>@1750rpm""",1750,"""15c<br>25h""",25,"""awd""","""8 Speed<br><em>Automatic/Manua…",108900,"""Audi RS 7"""
"""Germany""","""Audi""","""S7""",2016,"""Prestige quattro Hatchback""","""hatchback""","""450<br>@5800rpm""",5800,"""406<br>@1400rpm""",1400,"""17c<br>27h""",27,"""awd""","""7 Speed<br><em>Automatic</em>""",82900,"""Audi S7"""
"""Germany""","""Audi""","""S6""",2016,"""Premium Plus quattro Sedan""","""sedan""","""450<br>@5800rpm""",5800,"""406<br>@1400rpm""",1400,"""18c<br>27h""",27,"""awd""","""7 Speed<br><em>Automatic</em>""",70900,"""Audi S6"""


In [21]:
(
    GT(gtcars_final, groupname_col="ctry_origin", rowname_col="c_label")
    .cols_hide(
        ["mfr", "model", "drivetrain", "bdy_style", "mpg_h", "hp_rpm", "trq_rpm"]
    )
    .cols_move(["trsmn", "mpg_c"], after="trim")
    .tab_spanner(label="Performance", columns=["mpg_c", "hp", "trq"])
    .cols_label(**col_labels)
    .fmt_currency("msrp", decimals=0)
    .cols_align(align="center", columns=["mpg_c", "hp", "trq"])
    .tab_style(
        style=style.text(size=px("12")),
        locations=loc.body(columns=["trim", "trsmn", "mpg_c", "hp", "trq"]),
    )
    .tab_header(
        title=md("The Cars of **gtcars**"), subtitle="These are some fine automobiles"
    )
    .tab_source_note(md("Source: Various pages within the Edmonds website."))
)

GT(_tbl_data=shape: (47, 16)
┌──────────────┬─────────────┬────────┬──────┬───┬────────────┬─────────────┬────────┬─────────────┐
│ ctry_origin  ┆ mfr         ┆ model  ┆ year ┆ … ┆ drivetrain ┆ trsmn       ┆ msrp   ┆ c_label     │
│ ---          ┆ ---         ┆ ---    ┆ ---  ┆   ┆ ---        ┆ ---         ┆ ---    ┆ ---         │
│ str          ┆ str         ┆ str    ┆ u16  ┆   ┆ str        ┆ str         ┆ u32    ┆ str         │
╞══════════════╪═════════════╪════════╪══════╪═══╪════════════╪═════════════╪════════╪═════════════╡
│ Germany      ┆ Audi        ┆ R8     ┆ 2015 ┆ … ┆ awd        ┆ 6 Speed<br> ┆ 115900 ┆ Audi R8     │
│              ┆             ┆        ┆      ┆   ┆            ┆ <em>Manual< ┆        ┆             │
│              ┆             ┆        ┆      ┆   ┆            ┆ /em>        ┆        ┆             │
│ Germany      ┆ Audi        ┆ S8     ┆ 2016 ┆ … ┆ awd        ┆ 8 Speed<br> ┆ 114900 ┆ Audi S8     │
│              ┆             ┆        ┆      ┆   ┆            ┆ <em>Automat ┆        ┆             │
│              ┆             ┆        ┆      ┆   ┆            ┆ ic/Manua…   ┆        ┆             │
│ Germany      ┆ Audi        ┆ RS 7   ┆ 2016 ┆ … ┆ awd        ┆ 8 Speed<br> ┆ 108900 ┆ Audi RS 7   │
│              ┆             ┆        ┆      ┆   ┆            ┆ <em>Automat ┆        ┆             │
│              ┆             ┆        ┆      ┆   ┆            ┆ ic/Manua…   ┆        ┆             │
│ Germany      ┆ Audi        ┆ S7     ┆ 2016 ┆ … ┆ awd        ┆ 7 Speed<br> ┆ 82900  ┆ Audi S7     │
│              ┆             ┆        ┆      ┆   ┆            ┆ <em>Automat ┆        ┆             │
│              ┆             ┆        ┆      ┆   ┆            ┆ ic</em>     ┆        ┆             │
│ Germany      ┆ Audi        ┆ S6     ┆ 2016 ┆ … ┆ awd        ┆ 7 Speed<br> ┆ 70900  ┆ Audi S6     │
│              ┆             ┆        ┆      ┆   ┆            ┆ <em>Automat ┆        ┆             │
│              ┆             ┆        ┆      ┆   ┆            ┆ ic</em>     ┆        ┆             │
│ …            ┆ …           ┆ …      ┆ …    ┆ … ┆ …          ┆ …           ┆ …      ┆ …           │
│ United       ┆ Jaguar      ┆ F-Type ┆ 2016 ┆ … ┆ rwd        ┆ 6 Speed<br> ┆ 65000  ┆ Jaguar      │
│ Kingdom      ┆             ┆        ┆      ┆   ┆            ┆ <em>Manual< ┆        ┆ F-Type      │
│              ┆             ┆        ┆      ┆   ┆            ┆ /em>        ┆        ┆             │
│ United       ┆ Lotus       ┆ Evora  ┆ 2017 ┆ … ┆ rwd        ┆ 6 Speed<br> ┆ 91900  ┆ Lotus Evora │
│ Kingdom      ┆             ┆        ┆      ┆   ┆            ┆ <em>Manual< ┆        ┆             │
│              ┆             ┆        ┆      ┆   ┆            ┆ /em>        ┆        ┆             │
│ United       ┆ McLaren     ┆ 570    ┆ 2016 ┆ … ┆ rwd        ┆ 7 Speed<br> ┆ 184900 ┆ McLaren 570 │
│ Kingdom      ┆             ┆        ┆      ┆   ┆            ┆ <em>Automat ┆        ┆             │
│              ┆             ┆        ┆      ┆   ┆            ┆ ic</em>     ┆        ┆             │
│ United       ┆ Rolls-Royce ┆ Dawn   ┆ 2016 ┆ … ┆ rwd        ┆ 8 Speed<br> ┆ 335000 ┆ Rolls-Royce │
│ Kingdom      ┆             ┆        ┆      ┆   ┆            ┆ <em>Automat ┆        ┆ Dawn        │
│              ┆             ┆        ┆      ┆   ┆            ┆ ic</em>     ┆        ┆             │
│ United       ┆ Rolls-Royce ┆ Wraith ┆ 2016 ┆ … ┆ rwd        ┆ 8 Speed<br> ┆ 304350 ┆ Rolls-Royce │
│ Kingdom      ┆             ┆        ┆      ┆   ┆            ┆ <em>Automat ┆        ┆ Wraith      │
│              ┆             ┆        ┆      ┆   ┆            ┆ ic</em>     ┆        ┆             │
└──────────────┴─────────────┴────────┴──────┴───┴────────────┴─────────────┴────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x7fb33e258640>, _boxhead=Boxhead([ColInfo(var='ctry_origin', type=<ColInfoTypeEnum.row_group: 3>, column_label='ctry_origin', column_align='left', column_width=None), ColInfo(var='mfr', ty